# Whisper-Notion Pipeline for Google Colab

## 사용 방법:
1. 이 노트북을 Google Colab에 업로드
2. **런타임 → 런타임 유형 변경 → GPU (T4)** 선택
3. 셀을 순서대로 실행

## 기능:
- 오디오 파일 전사 (GPU 가속)
- Gemini를 통한 요약
- 중간 결과 자동 저장 (체크포인트)
- Google Drive 연동

In [ ]:
# GPU 확인
!nvidia-smi
import torch
print(f"\nPyTorch GPU 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

In [ ]:
# 필요한 패키지 설치
!pip install -q openai-whisper
!pip install -q python-dotenv
!pip install -q pyyaml
!pip install -q rich

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 생성
import os
WORK_DIR = '/content/drive/MyDrive/whisper_notion_pipeline'
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(f'{WORK_DIR}/input', exist_ok=True)
os.makedirs(f'{WORK_DIR}/output', exist_ok=True)
os.makedirs(f'{WORK_DIR}/checkpoints', exist_ok=True)

print(f"작업 디렉토리: {WORK_DIR}")

In [ ]:
# 설정 클래스
from dataclasses import dataclass
from typing import Optional, List, Dict
import json
import time
from pathlib import Path

@dataclass
class Config:
    # Whisper 설정
    whisper_model: str = "large"  # tiny, base, small, medium, large, large-v3
    whisper_language: str = "ko"  # 한국어
    
    # 요약 설정
    enable_summary: bool = True
    summary_prompt: str = """다음 음성 녹음 내용을 요약해주세요:
    1. 핵심 논의 사항을 bullet point로 정리
    2. 중요한 결정사항이나 합의사항 명시
    3. 액션 아이템 정리 (담당자, 기한 포함)
    4. 불필요한 내용은 제외하고 핵심만 간결하게"""
    
    # 체크포인트 설정
    checkpoint_interval: int = 300  # 5분마다 저장
    
config = Config()

In [ ]:
# Whisper 전사 클래스
import whisper
import threading

class WhisperTranscriber:
    def __init__(self, model_name="base", device="cuda"):
        self.model_name = model_name
        self.device = device
        self.model = None
        self._checkpoint_lock = threading.Lock()
        
    def load_model(self):
        if self.model is None:
            print(f"Loading {self.model_name} model on {self.device}...")
            self.model = whisper.load_model(self.model_name, device=self.device)
            print("Model loaded!")
    
    def save_checkpoint(self, audio_path, segments, text, language):
        """체크포인트 저장"""
        checkpoint_path = f"{WORK_DIR}/checkpoints/{Path(audio_path).stem}_checkpoint.json"
        checkpoint_data = {
            "audio_path": str(audio_path),
            "language": language,
            "segments": segments,
            "text": text,
            "last_segment_time": segments[-1]["end"] if segments else 0,
            "timestamp": time.time()
        }
        
        with self._checkpoint_lock:
            with open(checkpoint_path, "w", encoding="utf-8") as f:
                json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)
        
        print(f"✓ 체크포인트 저장: {len(segments)} 세그먼트, {len(text)} 글자")
    
    def load_checkpoint(self, audio_path):
        """체크포인트 불러오기"""
        checkpoint_path = f"{WORK_DIR}/checkpoints/{Path(audio_path).stem}_checkpoint.json"
        if os.path.exists(checkpoint_path):
            with open(checkpoint_path, "r", encoding="utf-8") as f:
                return json.load(f)
        return None
    
    def clear_checkpoint(self, audio_path):
        """체크포인트 삭제"""
        checkpoint_path = f"{WORK_DIR}/checkpoints/{Path(audio_path).stem}_checkpoint.json"
        if os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)
            print("✓ 체크포인트 삭제됨")
    
    def transcribe(self, audio_path, language="ko"):
        """오디오 파일 전사"""
        self.load_model()
        
        # 체크포인트 확인
        checkpoint = self.load_checkpoint(audio_path)
        if checkpoint:
            print(f"✓ 체크포인트 발견: {len(checkpoint['segments'])} 세그먼트 복원")
        
        # 파일 크기 확인
        file_size_mb = os.path.getsize(audio_path) / 1024 / 1024
        print(f"\n파일: {Path(audio_path).name}")
        print(f"크기: {file_size_mb:.1f}MB")
        print(f"모델: {self.model_name}")
        print(f"디바이스: {self.device}")
        
        # 전사 시작
        print("\n전사 시작...")
        start_time = time.time()
        
        result = self.model.transcribe(
            audio_path,
            language=language,
            fp16=(self.device == "cuda"),  # GPU에서만 FP16 사용
            verbose=True,
            task="transcribe"
        )
        
        elapsed_time = time.time() - start_time
        print(f"\n✓ 전사 완료!")
        print(f"  - 소요 시간: {elapsed_time:.1f}초")
        print(f"  - 언어: {result['language']}")
        print(f"  - 텍스트 길이: {len(result['text'])} 글자")
        
        # 최종 체크포인트 저장 후 삭제
        if result.get('segments'):
            self.save_checkpoint(audio_path, result['segments'], result['text'], result['language'])
        self.clear_checkpoint(audio_path)
        
        return result

In [ ]:
# 요약 클래스 (Gemini CLI 사용)
import subprocess

class Summarizer:
    def __init__(self):
        # Gemini CLI 설치 확인
        try:
            subprocess.run(["gemini", "--version"], capture_output=True, check=True)
            self.has_gemini = True
            print("✓ Gemini CLI 사용 가능")
        except:
            self.has_gemini = False
            print("✗ Gemini CLI 없음 - 요약 기능 비활성화")
            print("  설치하려면: npm install -g @google/generative-ai-cli")
    
    def summarize(self, text, prompt=None):
        """텍스트 요약"""
        if not self.has_gemini:
            return None
        
        if prompt is None:
            prompt = config.summary_prompt
        
        full_prompt = f"{prompt}\n\n{text}"
        
        try:
            print("\n요약 생성 중...")
            result = subprocess.run(
                ["gemini"],
                input=full_prompt,
                capture_output=True,
                text=True,
                check=True
            )
            summary = result.stdout.strip()
            print("✓ 요약 완료")
            return summary
        except Exception as e:
            print(f"✗ 요약 실패: {e}")
            return None

In [ ]:
# 파이프라인 실행 함수
def process_audio_file(audio_path, model_name="base"):
    """오디오 파일 처리 파이프라인"""
    audio_path = Path(audio_path)
    if not audio_path.exists():
        print(f"❌ 파일을 찾을 수 없습니다: {audio_path}")
        return None
    
    # 출력 디렉토리 생성
    output_dir = Path(f"{WORK_DIR}/output/{audio_path.stem}")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n{'='*60}")
    print(f"처리 시작: {audio_path.name}")
    print(f"{'='*60}")
    
    # 1. 전사
    transcriber = WhisperTranscriber(model_name=model_name)
    result = transcriber.transcribe(audio_path, language=config.whisper_language)
    
    # 전사 결과 저장
    with open(output_dir / "transcript.txt", "w", encoding="utf-8") as f:
        f.write(result['text'])
    
    # SRT 자막 저장
    save_srt(result['segments'], output_dir / "transcript.srt")
    
    # JSON 저장
    with open(output_dir / "transcript.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ 전사 결과 저장: {output_dir}")
    
    # 2. 요약 (선택적)
    if config.enable_summary:
        summarizer = Summarizer()
        summary = summarizer.summarize(result['text'])
        
        if summary:
            with open(output_dir / "summary.md", "w", encoding="utf-8") as f:
                f.write(f"# 요약: {audio_path.name}\n\n")
                f.write(f"- 언어: {result['language']}\n")
                f.write(f"- 길이: {len(result['text'])} 글자\n\n")
                f.write("## 요약 내용\n\n")
                f.write(summary)
            print(f"✓ 요약 저장: {output_dir / 'summary.md'}")
    
    print(f"\n{'='*60}")
    print(f"✅ 처리 완료: {audio_path.name}")
    print(f"{'='*60}\n")
    
    return result

def save_srt(segments, output_path):
    """SRT 자막 파일 저장"""
    with open(output_path, "w", encoding="utf-8") as f:
        for i, segment in enumerate(segments, 1):
            start = format_timestamp(segment['start'])
            end = format_timestamp(segment['end'])
            text = segment['text'].strip()
            f.write(f"{i}\n{start} --> {end}\n{text}\n\n")

def format_timestamp(seconds):
    """초를 SRT 타임스탬프 형식으로 변환"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

## 파일 업로드 및 처리

### 옵션 1: 직접 업로드 (작은 파일)

In [ ]:
# 파일 직접 업로드
from google.colab import files

uploaded = files.upload()
if uploaded:
    # 업로드된 파일을 input 폴더로 이동
    for filename in uploaded.keys():
        input_path = f"{WORK_DIR}/input/{filename}"
        with open(input_path, "wb") as f:
            f.write(uploaded[filename])
        print(f"✓ 파일 저장: {input_path}")
        
        # 처리 실행
        process_audio_file(input_path, model_name="base")

### 옵션 2: Google Drive에서 처리

In [ ]:
# Drive의 input 폴더에서 오디오 파일 찾기
import glob

audio_extensions = ['*.mp3', '*.m4a', '*.wav', '*.flac', '*.ogg']
audio_files = []

for ext in audio_extensions:
    audio_files.extend(glob.glob(f"{WORK_DIR}/input/{ext}"))

print(f"발견된 오디오 파일: {len(audio_files)}개")
for i, f in enumerate(audio_files):
    print(f"{i+1}. {Path(f).name}")

In [ ]:
# 특정 파일 처리 (파일 번호 선택)
if audio_files:
    # 처리할 파일 선택 (1번 파일 처리하려면 0 입력)
    file_index = 0  # 변경하세요
    
    if 0 <= file_index < len(audio_files):
        selected_file = audio_files[file_index]
        process_audio_file(selected_file, model_name="base")  # 모델 선택 가능
    else:
        print("❌ 잘못된 파일 번호")

In [ ]:
# 모든 파일 일괄 처리
for audio_file in audio_files:
    try:
        process_audio_file(audio_file, model_name="base")
    except Exception as e:
        print(f"❌ 오류 발생 ({Path(audio_file).name}): {e}")
        continue

## 결과 다운로드

In [ ]:
# 결과 압축 및 다운로드
import shutil
from datetime import datetime

# 출력 폴더 압축
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f"whisper_results_{timestamp}"
shutil.make_archive(archive_name, 'zip', f"{WORK_DIR}/output")

# 다운로드
files.download(f"{archive_name}.zip")
print(f"✓ 다운로드 완료: {archive_name}.zip")

In [ ]:
# 설정 변경 예시

# 1. 모델 변경 (더 정확한 결과)
config.whisper_model = "small"  # 또는 "medium", "large-v3"

# 2. 언어 변경
config.whisper_language = "en"  # 영어
# config.whisper_language = None  # 자동 감지

# 3. 요약 프롬프트 커스터마이징
config.summary_prompt = """
Please summarize this transcript in English:
- Key points in bullet format
- Important decisions
- Action items with owners
"""

print("✓ 설정 변경 완료")